# Creación de la arquitectura encoder-decoder y función de implementación de la App EmpAI

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate -U
!pip install transformers[torch]
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (

In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl 

In [3]:
%%capture
!pip install fairseq2
!pip install pydub sentencepiece
!pip install git+https://github.com/facebookresearch/seamless_communication.git

In [4]:
import io
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import mmap
import numpy
import soundfile
import torchaudio
import torch

from collections import defaultdict
from IPython.display import Audio, display
from pathlib import Path
from pydub import AudioSegment

from seamless_communication.inference import Translator
from seamless_communication.streaming.dataloaders.s2tt import SileroVADSilenceRemover

Prueba con template chat

In [5]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [6]:
!git config --global user.email "luisangel.motta@alumnos.viu.es"
!git config --global user.name "AngelMV97"

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

model_path = "LuangMV97/cm-bert_EmpAI"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path, ignore_mismatched_sizes=True)

classifier = pipeline(task="sentiment-analysis", model=model, tokenizer=tokenizer)

In [8]:
# Initialize a Translator object with a multitask model, vocoder on the GPU.

model_name = "seamlessM4T_v2_large"
vocoder_name = "vocoder_v2" if model_name == "seamlessM4T_v2_large" else "vocoder_36langs"

translator = Translator(
    model_name,
    vocoder_name,
    device=torch.device("cuda:0"),
    dtype=torch.float16,
)

100%|██████████| 8.45G/8.45G [00:54<00:00, 168MB/s] 
100%|██████████| 360k/360k [00:00<00:00, 52.6MB/s]
100%|██████████| 4.93M/4.93M [00:00<00:00, 143MB/s]
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
100%|██████████| 160M/160M [00:00<00:00, 239MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [9]:
def translate_inference(file_path, task, src_lang, tgt_lang="eng"):

  text_output, speech_output = translator.predict(
        input=file_path,
        task_str=task,
        tgt_lang=tgt_lang,
        src_lang=src_lang,
    )
  if task == "t2st":
    return speech_output

  return text_output

In [10]:
import torch

class DialogueGenerator(torch.nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, messages):
        self.messages = messages
        # Encode the input text
        input_ids = encoder_tokenizer.apply_chat_template(self.messages,
                                                          tokenize=True,
                                                          add_generation_prompt=True,
                                                          return_tensors="pt")
        input_length = input_ids.shape[1]
        attention_mask = torch.ones_like(input_ids)

        # Generate the response
        outputs = self.decoder.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=75,
            num_return_sequences=1,
            do_sample=True,
            top_k=5,
            top_p=0.9
        )

        # Decode the generated response
        response = decoder_tokenizer.batch_decode(outputs[:, input_length:], skip_special_tokens=True)[0]

        return response

In [11]:
import random
def EmpAIGenerator(input_text:str, encoder_name:str, decoder_name:str, language='eng'):
    umbral_sentiment = 2
    sentiment_intensity = classifier(input_text)
    sentiment_intensity = sentiment_intensity[0]['score']
    modalidad = random.choice('texto', 'audio')

    if '/' in input_text: # input_text es unla ruta a un archivo (audio)
        task = 's2tt'
    else:
        task = 't2tt'
    if language != 'eng':
        input_text = translate_inference(input_text, task, src_lang=language)
    elif task == 's2tt':
        input_text = translate_inference(input_text, task, src_lang=language)

    # Carga el tikenizador
    encoder_tokenizer = RobertaTokenizer.from_pretrained(encoder_name, padding_side='left')
    decoder_tokenizer = GPT2Tokenizer.from_pretrained(decoder_name)
    # Carga los modelo entrenados RoBERTa and GPT2
    encoder = RobertaForMaskedLM.from_pretrained(encoder_name)
    decoder = GPT2LMHeadModel.from_pretrained(decoder_name, pad_token_id=decoder_tokenizer.eos_token_id)
    # Crea el modelo generador de diálogo
    model = DialogueGenerator(encoder, decoder)

    # Genera la respuesta
    messages = [{"role": "user", "content": input_text}]
    response = model(messages)

    if language != 'eng':
        response = translate_inference(response, task='t2tt', src_lang='eng', tgt_lang=language)

    if sentiment_intensity > umbral_sentiment or modalidad == 'audio':
        audio_response = translate_inference(response, task='t2st', src_lang=language, tgt_lang=language)

        return audio_response
    else:
        return response

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, GPT2LMHeadModel, GPT2Tokenizer

# Load the tokenizers
encoder_name = "LuangMV97/MentalRoBERTa_EmpAI_final"
decoder_name = "LuangMV97/DialoGPT_EmpAI_final_FineTuned"
#encoder_name = "LuangMV97/RoBERTa_EmpAI_final"
#decoder_name = "LuangMV97/GPT2_EmpAI"
encoder_tokenizer = RobertaTokenizer.from_pretrained(encoder_name, padding_side='left')
decoder_tokenizer = GPT2Tokenizer.from_pretrained(decoder_name)

# Load the pre-trained RoBERTa and GPT2 models
encoder = RobertaForMaskedLM.from_pretrained(encoder_name)
decoder = GPT2LMHeadModel.from_pretrained(decoder_name, pad_token_id=decoder_tokenizer.eos_token_id)

# Create the dialogue generator model
model = DialogueGenerator(encoder, decoder)

# Example usage
#"Hi, I need to talk to someone who can give me psychological and emotional support. "
input_text = "Yesterday I had an argument with my father and I feel bad about it, what should I do?"
messages = [
    {
        "role": "system",
        "content": "You are an counseling chatbot who understands the user and offers psychological support."
    },
    {"role": "user", "content": input_text},
]
response = model(messages)

print(f"Input: {input_text}")
print(f"Response: {response}")

Input: Yesterday I had an argument with my father and I feel bad about it, what should I do?
Response: !! I feel very comfortable in my new life and have been doing well in my work. I am very happy.
I feel very comfortable with my new life. I feel like I have a great relationship with my new wife! 
I am so happy I have such a nice wife! 

How's your wife doing these days? 
I am


In [ ]:
input_text = "I can't belive you are not usefull."
messages = [{"role": "user", "content": input_text}]

response = model(messages)

print(f"Input: {input_text}")
print(f"Response: {response}")

Input: I can't belive you are not usefull.
Response: ationagain
I am in my early 20s. I'm in a relationship with this guy who has been around for a long time and he is a great friend. I feel so alone and so alone.
I feel very alone and I am very scared. I have no friends. I don't know what to do. I feel like nothing.
I feel like


In [40]:
import gradio as gr

title = "EmpAI"
description = """
Hola, soy tu asistente virtual de EmpAI.
<br>
<img src='https://r2.easyimg.io/j0kn15r8c/logorelleno.png' width=100px>
¿Que puedo hacer por ti?
"""

demo = gr.Interface(fn=EmpAIGenerator,
                    inputs=[
                        gr.Audio(type="numpy", label="Speak here..."),
                        gr.Textbox(label="Text here..."),
                        gr.Textbox(value='spa', label="Desired language")
                    ],
                    outputs="text",
                    title=title,
                    description=description,
                    )

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e8cd539ddd272ce0f3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Arquitectura encoder-decoder construida

In [ ]:
input_text = "I'm feeling a bit down today."
response = DialogueGenerator(encoder, decoder)

print(f"Input: {input_text}")
print(f"Response: {response}")

Input: I'm feeling a bit down today.
Response: DialogueGenerator(
  (encoder): RobertaForMaskedLM(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): Rober

In [ ]:
input_text = "I'm feeling a bit down today."
response = DialogueGenerator(encoder, decoder)

print(f"Input: {input_text}")
print(f"Response: {response}")

Input: I'm feeling a bit down today.
Response: DialogueGenerator(
  (encoder): RobertaForMaskedLM(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): Rober